<a href="https://colab.research.google.com/github/carinadourado/carinadourado/blob/main/machine_learning/aula_6_modelo_teachable_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.insper.edu.br/wp-content/themes/insper/dist/image/logo.png" width="80">. 

## Machine Learning Aplicado ao Jornalismo
Aula 6: Usando o [Teachable Machine](https://teachablemachine.withgoogle.com/) do Google para reconhecer a presença de pessoas em determinadas imagens.

Este código presume que você já capturou uma grande quantidade de imagens. Para mais detalhes do experimento com a raspagem do Instagram, leia o outro notebook, disponibilizado no Blackboard (é necessário rodá-lo localmente).

In [ ]:
# Para facilitar o trabalho com arquivos, "montamos" o Google Drive desta forma:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Importamos as bibliotecas Keras (Uma biblioteca open-source de rede neural), que trabalha em cima do TensorFlow
# PIL é uma biblioteca que faz diversas operações com imagens
# Numpy é usada para representar e manipular elementos matemáticos como matrizes e "data series"

from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Carregamos o modelo que salvamos usando o https://teachablemachine.withgoogle.com/train/image
model = load_model('/content/gdrive/MyDrive/Master em Jornalismo de Dados 2021/Machine Learning Aplicado ao Jornalismo/arquivos_notebook_ml_insta/keras_model.h5', compile=False)

Agora testamos o modelo com fotos armazenadas no Google Drive. 

📁 Navegue na barra lateral, clique na pasta onde estão as fotos e depois clique em . . . ➡️ `"Copiar caminho"`. Este é o endereço que você usará no código seguinte.  

In [ ]:
# Criamos uma matriz de 3 dimensões, de 224x224 para armazenar a foto. O primeiro número é a quantidade de imagens. 
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Abrimos a imagem (lembre de mudar o caminho para o endereço da imagem no seu drive, ou em algum lugar na web)
caminho = '/content/gdrive/MyDrive/Master em Jornalismo de Dados 2021/Machine Learning Aplicado ao Jornalismo/arquivos_notebook_ml_insta/imagens/minfraestrutura_84.jpeg'
image = Image.open(caminho)

# Convertemos a imagem para "RGB"
image = image.convert('RGB')

# Depois mudamos o seu tamanho para 224x224 pixels, e recortando o centro para ficar um quadrado
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

# Transformamos a imagem em uma matriz do Numpy
image_array = np.asarray(image)

# Normalizamos a imagem
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Colocamos a imagem na matriz criada no início
data[0] = normalized_image_array

# Rodamos o modelo
prediction = model.predict(data)
print(prediction)


[[6.9890276e-04 9.9930108e-01]]


O Que esta "previsão" significa? Olhando a imagem em questão:



![](https://drive.google.com/uc?id=1FX5sD2dmrZVsIqX64-D8geggIBFMtZHl)

O preditor estimou que o a chance de a imagem ser da categoria "0" (Tarcisio) era significativamente maior do que a de ser não-Tarcisio "1". Como há apenas duas categorias, o preditor irá dar o resultado em forma de matriz com dois elementos. O primeiro é a chance para a categoria 0, o segundo é a categoria 1

In [ ]:
# Uma maneira de simplificar o código é usar esta função.
# Ela já diz qual é a categoria com maior probabilidade, de acordo com o preditor
np.argmax(prediction)

1

In [ ]:
# Agora fazemos um loop para rodar o preditor em cada uma das imagens
resultados = {0: "Ministro", 1: "Outras imagens"}

def predict_ministro(imagem):
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
  image = Image.open(imagem)
  image = image.convert('RGB')
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.ANTIALIAS)
  image_array = np.asarray(image)
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  data[0] = normalized_image_array
  prediction = model.predict(data)
  categoria = np.argmax(prediction)
  arquivo = imagem.split("/")[-1] # Pegamos só o nome do arquivo, não o caminho completo
  dicio = {"resultado": resultados[categoria], "imagem": arquivo, "confiança": prediction.max()}
  return dicio

In [ ]:
# Testamos a função com uma imagem
predict_ministro("/content/gdrive/MyDrive/Master em Jornalismo de Dados 2021/Machine Learning Aplicado ao Jornalismo/arquivos_notebook_ml_insta/imagens/minfraestrutura_79.jpeg")

{'confiança': 0.86160314,
 'imagem': 'minfraestrutura_79.jpeg',
 'resultado': 'Ministro'}

In [ ]:
# Agora criamos um loop. O endereço das imagens é igual salvo o número antes do .jpeg
dados = []
for i in range(1,109):
  img = f'/content/gdrive/MyDrive/Master em Jornalismo de Dados 2021/Machine Learning Aplicado ao Jornalismo/arquivos_notebook_ml_insta/imagens/minfraestrutura_{i}.jpeg'
  predicao = predict_ministro(img)
  dados.append(predicao)

In [ ]:
dados

[{'confiança': 0.9999701,
  'imagem': 'minfraestrutura_1.jpeg',
  'resultado': 'Ministro'},
 {'confiança': 0.9991399,
  'imagem': 'minfraestrutura_2.jpeg',
  'resultado': 'Outras imagens'},
 {'confiança': 0.85392046,
  'imagem': 'minfraestrutura_3.jpeg',
  'resultado': 'Outras imagens'},
 {'confiança': 0.99999654,
  'imagem': 'minfraestrutura_4.jpeg',
  'resultado': 'Outras imagens'},
 {'confiança': 0.9989095,
  'imagem': 'minfraestrutura_5.jpeg',
  'resultado': 'Ministro'},
 {'confiança': 0.99996173,
  'imagem': 'minfraestrutura_6.jpeg',
  'resultado': 'Outras imagens'},
 {'confiança': 0.99995303,
  'imagem': 'minfraestrutura_7.jpeg',
  'resultado': 'Ministro'},
 {'confiança': 0.961404,
  'imagem': 'minfraestrutura_8.jpeg',
  'resultado': 'Ministro'},
 {'confiança': 0.99974054,
  'imagem': 'minfraestrutura_9.jpeg',
  'resultado': 'Outras imagens'},
 {'confiança': 0.9995696,
  'imagem': 'minfraestrutura_10.jpeg',
  'resultado': 'Ministro'},
 {'confiança': 0.99959844,
  'imagem': 'minfr

In [ ]:
# Criamos um dataframe para ficar mais fácil enxergar os dados
import pandas as pd

df = pd.DataFrame(dados)

In [ ]:
# E este é o resultado da nossa investigação
df['resultado'].value_counts()

Outras imagens    67
Ministro          41
Name: resultado, dtype: int64

In [ ]:
# Agora baixamos os dados para checar manualmente se o modelo está prevendo bem.
from google.colab import files

df.to_csv("resultados_tarcisio.csv")
files.download('resultados_tarcisio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>